In [ ]:
from nerfstudio.utils.eval_utils import eval_setup
from nerfstudio.cameras.camera_paths import get_interpolated_camera_path
from pathlib import Path
from IPython.display import display, clear_output
import torch
import numpy as np
import src.ddpg as ddpg
import src.sac as sac
import src.shared as shared

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
downscale = 32

In [ ]:
config_path = Path(f'nerfstudio/lego/nerfacto/2024-05-02_094606/config-down-x{downscale}.yml')
db_path = f'nerfstudio/db-x{downscale}.pkl'

_, pipeline, _, _ = eval_setup(config_path, test_mode='val')

train_dataset = pipeline.datamanager.train_dataset
eval_dataset = pipeline.datamanager.eval_dataset
model = pipeline.model

In [ ]:
# pre-render NeRF
try:
    db = torch.load(db_path)
except:
    train_loader = shared.CustomDL(train_dataset.cameras, device=device)
    test_loader = shared.CustomDL(eval_dataset.cameras, device=device)

    db = dict()
    shared.nerf.pre_render(model, train_loader, db)
    shared.nerf.pre_render(model, test_loader, db)
    torch.save(db, db_path)

In [ ]:
# fetch image size
db0 = db[list(db.keys())[0]]
imsize = db0['imsize']
imshape = db0['imshape']
f'imsize = {imsize}, imshape = {imshape}'

In [ ]:
custom_nerf = shared.nerf.CustomNeRF(db)

In [ ]:
try:
    dist_mat = torch.load('nerfstudio/dist_mat.pkl')
except:
    dl1 = shared.CustomDL(train_dataset.cameras, device=device)
    dl2 = shared.CustomDL(train_dataset.cameras, device=device)

    dist_mat = torch.full((len(dl1), len(dl2)), torch.inf)

    for i, rb1 in enumerate(dl1):
        oi = rb1.origins[0]
        dl2.reset(dl1.step)

        for j, rb2 in enumerate(dl2, start=i+1):
            display(f'{i} {j}')
            clear_output(wait=True)
            oj = rb2.origins[0]

            dist = (oi - oj).norm().item()
            dist_mat[i, j] = dist

    torch.save(dist_mat, 'nerfstudio/dist_mat.pkl')

In [ ]:
# sort camera pairs by ascending distance
k = 6800

dist_flat = dist_mat.flatten()
top_val, idx_flat = dist_flat.topk(k, largest=False)

row_idx = idx_flat // dist_mat.size(1)
col_idx = idx_flat % dist_mat.size(1)
top_idx = torch.stack((row_idx, col_idx), dim=1)

In [ ]:
top_idx, top_val

In [ ]:
# experiment 1: far vs close camera pair
idx = 43 # 0.06

In [ ]:
idx = 155 # 0.12

In [ ]:
idx = 560 # 0.24

In [ ]:
idx = 1995 # 0.48

In [ ]:
idx = 6786 # 0.96

In [ ]:
top_idx[idx], top_val[idx]

In [ ]:
camera_pair = train_dataset.cameras[top_idx[idx]]
camera_interpol = get_interpolated_camera_path(camera_pair, steps=3, order_poses=True)[1:2]

train_loader = shared.CustomDL(camera_pair, step_size=len(camera_pair), device=device, disable_distortion=True)
test_loader = shared.CustomDL(camera_interpol, step_size=len(camera_interpol), device=device, disable_distortion=True)

In [ ]:
# DDPG
config = ddpg.Config(
    warmup=5000,
    max_steps=True,
    eval_episodes=1,
)

rad_thres = .5
rad_goal = .5

env = shared.env.NerfEnv(
    custom_nerf,
    rad_thres=rad_thres,
    rad_goal=rad_goal,
    reward_scale=10,
    reward_max_resolution=4,
    obscure=True,
)

runner = ddpg.Runner(
    train_loader,
    test_loader,
    env,
    config,
    tensorboard=True,
    with_img=True,
    # save_path='out/ddpg-0.06',
)

In [ ]:
# SAC
config = sac.Config(
    warmup=1000,
    mem_batch_size=256,
    max_steps=True,
    eval_episodes=1,
)

rad_thres = .5
rad_goal = .5

env = shared.env.NerfEnv(
    custom_nerf,
    rad_thres=rad_thres,
    rad_goal=rad_goal,
    reward_scale=10,
    reward_max_resolution=4,
)

runner = sac.Runner(
    train_loader,
    test_loader,
    env,
    config,
    tensorboard=True,
    with_img=True,
    # save_path='out/sac-0.06',
)

In [ ]:
runner(mode='train')

In [ ]:
runner.env.model = model
runner.test_loader.random_rays = False

intgr = runner(mode='test', load_weights={
    'actor': f'{runner.save_path}/actor.pkl',
    'critic': f'{runner.save_path}/critic.pkl'
})

In [ ]:
print(intgr)

In [ ]:
plot_loader = shared.CustomDL(camera_pair, step_size=2, device=device, disable_distortion=True)
fname = 'camera-pose'
goal = False

In [ ]:
fname = 'camera-pose-goal'
goal = True

In [ ]:
plot_loader = shared.CustomDL(camera_interpol, step_size=1, device=device, disable_distortion=True)
fname = 'interpol'
goal = False

In [ ]:
fname = 'interpol-goal'
goal = True

In [ ]:
for rb in plot_loader:
    rb.directions = shared.geom.camera_to_world(rb.directions, rb.camera_to_world) - rb.origins
    out = model(rb)
    rgb = out['rgb']
    gs = rgb.mean(dim=-1)
    if goal:
        gs[gs < rad_goal] = 0
    img = gs.reshape(-1, *rb.imshape).detach().cpu().numpy()
    shared.utils.show_img(img, title=fname, stdout=True, write_path=runner.save_path)

In [ ]:
# statistical test: naive Monte Carlo
test_loader.random_rays = True

iters = 1500
integrals = []

for _ in range(iters):
    for rb in test_loader:
        # convert from camera to world coords
        rb.directions = shared.geom.camera_to_world(
            rb.directions,
            rb.camera_to_world,
        ) - rb.origins

        out = model(rb)
        rgb = out['rgb']
        gs = rgb.mean(dim=-1).detach().cpu().numpy()

        goal = gs[gs >= rad_thres]
        size = goal.shape[0]

        intgr = (goal * size).mean()
        integrals.append(intgr)

integrals = np.array(integrals)
print(f'mean = {integrals.mean()}, std = {integrals.std()}')

In [ ]:
# statistical test: DDPG
runner.evaluator.tensorboard = False
runner.evaluator.with_img = False
runner.test_loader.random_rays = True
weights = {
    'actor': f'{runner.save_path}/actor.pkl',
    'critic': f'{runner.save_path}/critic.pkl'
}

iters = 10
integrals = []

for _ in range(iters):
    intgr = runner(mode='test', load_weights=weights, with_penalty=True)
    integrals.append(intgr)

integrals = np.array(integrals)
print(f'mean = {integrals.mean()}, std = {integrals.std()}')

In [ ]:
# experiment 2: camera with 3 neighbors (at different distances)
idx = torch.where(top_idx[:, 0] == top_idx[3, 0])[0]
idx_lst = list(set(top_idx[idx].flatten().tolist()))
idx_lst.remove(top_idx[3, 0].item())
train_idx = torch.tensor(idx_lst)
test_idx = top_idx[3, 0].unsqueeze(-1)

camera_train = train_dataset.cameras[train_idx]
camera_test = train_dataset.cameras[test_idx]

top_idx[idx], top_val[idx], train_idx, test_idx

In [ ]:
train_loader = shared.CustomDL(camera_train, step_size=len(train_idx), device=device)
test_loader = shared.CustomDL(camera_test, step_size=len(test_idx), device=device)

In [ ]:
plot_loader = shared.CustomDL(camera_train, step_size=len(train_idx), device=device)

In [ ]:
plot_loader = shared.CustomDL(camera_test, step_size=len(test_idx), device=device)

In [ ]:
for rb in plot_loader:
    directions = rb.directions.clone()
    rb.directions = shared.geom.camera_to_world(rb.directions, rb.camera_to_world) - rb.origins
    out = custom_nerf(rb)
    ref = model(rb)
    rb.directions = directions
    rgb1 = out['rgb']
    gs1 = rgb1.mean(dim=-1)
    img1 = gs1.reshape(rgb1.shape[0] // rb.imsize, *rb.imshape).detach().cpu().numpy()
    shared.utils.show_img(img1, title='bilinear interpolation', stdout=True)
    rgb2 = ref['rgb']
    gs2 = rgb2.mean(dim=-1)
    img2 = gs2.reshape(rgb2.shape[0] // rb.imsize, *rb.imshape).detach().cpu().numpy()
    shared.utils.show_img(img2, title='NeRF', stdout=True)

In [ ]:
# entire dataset
train_loader = shared.CustomDL(train_dataset.cameras, step_size=10, device=device)
test_loader = shared.CustomDL(eval_dataset.cameras, step_size=10, device=device)